In [1]:
#抓取百度数据
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
import pandas as pd
import re
import os

from datetime import datetime,timedelta
thedate=datetime.strftime(datetime.today()-timedelta(days=1),"%Y-%m-%d")
thedate2=datetime.strftime(datetime.today()-timedelta(days=1),"%m.%d")

browser = webdriver.Chrome()
url='https://motion.baidu.com/activity/yiqing_predict/main?province=%E5%85%A8%E5%9B%BD'
browser.get(url)

with open(r'./COVIDend/%s.html' % thedate,'w',encoding='utf-8') as f:
    f.write(browser.page_source)

browser.close()

In [2]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
from datetime import datetime,timedelta


city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])
city_popu.head(2)


from bs4 import BeautifulSoup
import lxml
# thedate="2022-12-30"
thedate=datetime.strftime(datetime.today()-timedelta(days=1),"%Y-%m-%d")

soup=BeautifulSoup(open(r'COVIDend\%s.html'  % thedate,'r',encoding='utf-8').read())

trs=soup.find_all('div',{"class":'list-item_1JNax'})

def get_result(tr):
    city=tr.find('div',{'class':'list-item-city_qLfui pc_UIzzf'}).find('div').text.replace('伊犁哈萨克自治州','伊犁')
    progress=tr.find('div',{'class':'rate-text_1PWuo'}).text
    peak=tr.find_all('div',{'class':'flex_3nTxm'})[1].text
    end=tr.find('div',{'class':'flex_3nTxm end_MnHhu'}).text
    return [city.replace('市',''),progress,peak,end]

d=[get_result(tr) for tr in trs]

df=pd.DataFrame(data=d,columns=['city','progress','peak','end'])
df=df.sort_values(by='end')

def covid_progress(s):
    s=s.replace('%','')
    if '回落至' in s:
        s=s.replace('回落至','')
    if '上升至' in s:
        s=s.replace('上升至','')
        s=200-int(s)
    return int(s)    
df['progress_2']=df['progress'].apply(covid_progress)
print(thedate)
df=df[['city','peak','progress_2']]

city99=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='99city')
c1=city99['city'].isin(df['city'])
# c2=df['city'].isin(city99['city'])
city_lack=city99[~c1]
city_lack['progress_2']=0
print(city_lack)
df=pd.concat([df,city_lack])

2023-01-08
      city        peak  progress_2
1709    东莞  2022/12/22           0
1859    中山  2022/12/20           0
1139  乌鲁木齐  2022/12/21           0
1769    佛山  2022/12/22           0
89      全国  2022/12/21           0
1019    南京  2022/12/20           0
2819    南充  2022/12/16           0
1229    南宁  2022/12/21           0
359     南昌  2022/12/20           0
2159    南阳  2022/12/17           0
419     合肥  2022/12/20           0
1169    宿迁  2022/12/21           0
479     常州  2022/12/20           0
119     广州  2022/12/21           0
959     廊坊  2022/12/12           0
1619    徐州  2022/12/20           0
2189    德阳  2022/12/15           0
839     成都  2022/12/15           0
1409    扬州  2022/12/21           0
2699    新乡  2022/12/14           0
989     杭州  2022/12/22           0
2429    柳州  2022/12/20           0
689     武汉  2022/12/15           0
1949    洛阳  2022/12/13           0
149     济南  2022/12/18           0
2399    淮安  2022/12/21           0
209     深圳  2022/12/22           0
1589    温

In [3]:
df=df.merge(city_popu,left_on='city',right_on='城市')
df['thedate']=thedate
df.to_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
print(thedate)
df.sort_values(by='progress_2')

2023-01-08


,city,peak,progress_2,城市,GDP,人均可支配收入,常住人口,thedate
0,日喀则,2022/12/24,0,日喀则,322,39992,16,2023-01-08
217,扬州,2022/12/21,0,扬州,6048,47202,264,2023-01-08
216,成都,2022/12/15,0,成都,17717,48593,1543,2023-01-08
215,德阳,2022/12/15,0,德阳,2404,39360,104,2023-01-08
214,徐州,2022/12/20,0,徐州,7320,37523,596,2023-01-08
...,...,...,...,...,...,...,...,...
110,丽江,2022/12/23,10,丽江,513,35667,29,2023-01-08
190,漳州,2022/12/27,10,漳州,4546,40008,93,2023-01-08
189,北海,2022/12/26,10,北海,1277,37956,99,2023-01-08
178,临沧,2022/12/26,12,临沧,821,30794,35,2023-01-08


In [17]:
",".join(list(df[df['progress_2']<=1]['city']))

'日喀则,镇江,巴中,攀枝花,广元,聊城,六盘水,丹东,荆州,通辽,盐城,嘉兴,株洲,河池,兰州,宁波,张掖,亳州,石家庄,梧州,崇左,昆明,保定,惠州,九江,上饶,萍乡,安庆,天津,桂林,沈阳,晋中,武威,阳泉,北京,克拉玛依,绥化,鸡西,七台河,淮北,白银,铜仁,宣城,庆阳,定西,晋城,吉安,怀化,承德,长治,包头,漯河,太原,运城,安顺,遵义,邵阳,黄山,岳阳,南平,哈密,连云港,咸阳,贺州,贵港,济宁,青岛,临汾,渭南,绍兴,宁德,泰州,枣庄,贵阳,泰安,林芝,玉林,清远,肇庆,衢州,无锡,驻马店,滨州,宜昌,台州,江门,唐山,衡水,东莞,中山,乌鲁木齐,佛山,南京,南充,南宁,南昌,南阳,合肥,宿迁,常州,广州,廊坊,徐州,德阳,成都,扬州,新乡,杭州,柳州,武汉,洛阳,济南,淮安,深圳,温州,湖州,滁州,珠海,绵阳,芜湖,苏州,衡阳,西安,赣州,邢台,邯郸,郑州,重庆,金华,长沙'

In [48]:
df.to_clipboard()

In [7]:
# wkbk=df.rename(columns={'peak':'搜索峰值日','end':'高峰结束日'})
# title=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='citylist')
# title.merge(wkbk[['city','progress','搜索峰值日','高峰结束日']],how='left',on='city')[['progress','搜索峰值日','高峰结束日']].to_clipboard(index=False)

In [8]:
# #COVID END date

# allpeople=df['常住人口'].sum()
# # allpeople
# df2=df.groupby('end').sum().reset_index()
# df2['人口cumsum']=df2['常住人口'].cumsum()
# df2['人口cumsum_per']=df2['人口cumsum']/allpeople
# df2['sum']=np.nan
# df2[['end','sum','人口cumsum_per']].to_clipboard(index=False,header=None)

In [15]:

from bs4 import BeautifulSoup
import lxml

def getpv(thedate):
    soup=BeautifulSoup(open(r'COVIDend\%s.html'  % thedate,'r',encoding='utf-8').read())

    trs=soup.find_all('div',{"class":'list-item_1JNax'})

    def get_result(tr):
        city=tr.find('div',{'class':'list-item-city_qLfui pc_UIzzf'}).find('div').text.replace('伊犁哈萨克自治州','伊犁')
        progress=tr.find('div',{'class':'rate-text_1PWuo'}).text
        peak=tr.find_all('div',{'class':'flex_3nTxm'})[1].text
        end=tr.find('div',{'class':'flex_3nTxm end_MnHhu'}).text
        return [city.replace('市',''),progress,peak,end]

    d=[get_result(tr) for tr in trs]

    df=pd.DataFrame(data=d,columns=['city','progress','peak','end'])
    df=df.sort_values(by='end')
    df=df.merge(city_popu,left_on='city',right_on='城市')

    def covid_progress(s):
        s=s.replace('%','')
        if '回落至' in s:
            s=s.replace('回落至','')
        if '上升至' in s:
            s=s.replace('上升至','')
            s=200-int(s)
        return int(s)    
    df['progress_2']=df['progress'].apply(covid_progress)

    def covid_status(s):
        status=""
        if '上升至' in s:
            s=s.replace('上升至','').replace('%','')
            if int(s) <=50:
                status='1上升至峰值的0-50%'
            elif 50< int(s) <=80:
                status='2上升至峰值的50-80%'
            elif int(s)>80:
                status='3上升至峰值的80-100%'
        if '回落至' in s:
            s=s.replace('回落至','').replace('%','')
            if int(s) <=1:
                status='9下降至峰值的0-1%'
            elif 1< int(s) <=10:
                status='8下降至峰值的1-10%'
            elif 10< int(s) <=30:
                status='7下降至峰值的10-30%'
            elif 30< int(s) <=50:
                status='6下降至峰值的30-50%'
            elif 50< int(s) <=80:
                status='5下降至峰值的50-80%'
            elif int(s)>80:
                status='4下降至峰值的80-100%'
        return status

    df['status']=df['progress'].apply(covid_status)
    df['date']=thedate



    pv_value=pd.pivot_table(df, values='常住人口', index=['date'],columns=["status"], aggfunc=np.sum)

    pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
    return pv_value

import os
final=pd.DataFrame()
for f in os.listdir(r'COVIDend'):
    thedate=f.replace('.html','')
    final=pd.concat([final,getpv(thedate)])

final['0 sum']=1
final=final.fillna(0)
final=final.sort_index(axis=1)
final.tail(22).to_clipboard(header=None)

final.tail(22)
# pv_value.tail(3)

status,0 sum,1上升至峰值的0-50%,2上升至峰值的50-80%,3上升至峰值的80-100%,4下降至至峰值的80-100%,5下降至峰值的50-80%,6下降至峰值的30-50%,7下降至峰值的10-30%,8下降至峰值的1-10%,9下降至峰值的0-1%
date,,,,,,,,,,
2022-12-18,1,0.035309,0.192308,0.373902,0.359497,0.038985,0.000000,0.000000,0.000000,0.000000
2022-12-19,1,0.035046,0.190876,0.371118,0.209385,0.182808,0.005858,0.004909,0.000000,0.000000
2022-12-20,1,0.002557,0.039172,0.314885,0.277620,0.317044,0.041094,0.007627,0.000000,0.000000
2022-12-21,1,0.000000,0.027279,0.225210,0.265670,0.413453,0.060755,0.007633,0.000000,0.000000
2022-12-22,1,0.000000,0.008640,0.136034,0.274860,0.431649,0.139434,0.006636,0.002747,0.000000
2022-12-23,1,0.000000,0.012012,0.119395,0.285921,0.286305,0.162700,0.130934,0.002733,0.000000
2022-12-24,1,0.000000,0.002805,0.108921,0.177414,0.333798,0.166179,0.202942,0.007940,0.000000
2022-12-25,1,0.000000,0.000000,0.047416,0.110329,0.358914,0.204460,0.265929,0.012952,0.000000
2022-12-26,1,0.000000,0.000000,0.017671,0.073696,0.276406,0.282346,0.303252,0.046629,0.000000
